In [ ]:
"""
Configuração inicial da aplicação Flask.

Este módulo cria a instância principal da aplicação Flask e define as configurações
necessárias para o uso de:

- Banco de dados (SQLite via SQLAlchemy)
- Controle de modificações do SQLAlchemy
- Autenticação baseada em JWT (JSON Web Tokens)

Essas configurações são essenciais para garantir o correto funcionamento da API REST.
"""

from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_jwt_extended import JWTManager, create_access_token, jwt_required
from datetime import datetime


# Inicialização do app
app = Flask(__name__)

# Configuração da URI de conexão com o banco de dados SQLite.
# O banco de dados será um arquivo chamado 'users.db' na raiz do projeto.
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'

# Desativa o rastreamento de modificações de objetos pelo SQLAlchemy.
# Essa configuração reduz o uso de memória e elimina warnings desnecessários.
# Recomenda-se manter como False, a menos que você precise explicitamente dessa funcionalidade.
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 

# Define a chave secreta usada para assinar os tokens JWT.
# Essa chave deve ser mantida em segredo, principalmente em ambiente de produção,
# pois garante a integridade e autenticidade dos tokens gerados.
app.config['JWT_SECRET_KEY'] = 'chave-super-secreta-rs'

db = SQLAlchemy(app)
jwt = JWTManager(app)

In [ ]:
class User(db.Model):
    """
    Modelo de usuário para a aplicação Flask, representando a tabela 'user' no banco de dados.

    Atributos:
        id (int): Identificador único do usuário (chave primária).
        nome (str): Nome do usuário (obrigatório).
        email (str): E-mail do usuário (único e obrigatório).
        created_at (datetime): Data e hora de criação do registro (preenchido automaticamente).
    """

    id = db.Column(db.Integer, primary_key=True)                    # ID único para cada usuário (Primary Key)
    nome = db.Column(db.String(80), nullable=False)                 # Nome do usuário (não pode ser nulo)
    email = db.Column(db.String(120), unique=True, nullable=False)  # E-mail único (não pode ser nulo)
    created_at = db.Column(db.DateTime, default=datetime.now)       # Timestamp automático de criação

    def __init__(self, nome, email):
        """
        Construtor da classe User.

        Args:
            nome (str): Nome do usuário.
            email (str): Endereço de e-mail do usuário.
        """
        self.nome = nome
        self.email = email

    def to_dict(self):
        """
        Converte o objeto User para um dicionário Python.

        Returns:
            dict: Representação do usuário como dicionário, útil para serialização em JSON.
        """
        return {
            'id': self.id,
            'nome': self.nome,
            'email': self.email,
            'created_at': self.created_at.isoformat()
        }


In [ ]:
# Cria o banco e a tabela
with app.app_context():
    db.create_all()

#### Treinamento e Salvamento de Modelo de Regressão Linear com Pickle

Este bloco cria um modelo de Regressão Linear usando `scikit-learn`, realiza o treinamento com dados fictícios e salva o modelo no formato `.pkl` usando a biblioteca `pickle`.

O modelo será utilizado posteriormente em uma rota da API Flask para fazer predições.

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pickle

X = np.array([[1], [2], [3], [4], [5]])  # Feature: número de horas
y = np.array([2, 4, 6, 8, 10])           # Target: nota final correspondente

model = LinearRegression()
model.fit(X, y)

with open('modelo_regressao.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Modelo treinado e salvo com sucesso como 'modelo_regressao.pkl'.")

# Carrega o modelo
with open('modelo_regressao.pkl', 'rb') as file:
    modelo_regressao = pickle.load(file)


In [ ]:
@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint de autenticação de usuário.

    Recebe um JSON com credenciais de login (usuário e senha) e retorna um token JWT
    caso as credenciais sejam válidas.

    Método: POST  
    URL: /login  

    Requisição JSON esperada:
        {
            "username": "admin",
            "password": "admin"
        }

    Repostas:
        200 OK:
            - Retorna um token JWT para autenticação nas demais rotas protegidas.
            Exemplo de resposta:
            {
                "access_token": "<jwt_token>"
            }

        401 Unauthorized:
            - Caso as credenciais estejam incorretas.
            Exemplo de resposta:
            {
                "msg": "Credenciais inválidas"
            }

    Observações:
        - Neste exemplo, o login é fixo: apenas o usuário "admin" com a senha "admin" pode autenticar.
        - Em aplicações reais, recomenda-se validar os dados com um banco de usuários.

    Returns:
        flask.Response: Objeto de resposta JSON contendo o token ou a mensagem de erro.
    """
    data = request.get_json()
    if data.get('username') == 'admin' and data.get('password') == 'admin':
        token = create_access_token(identity='admin')
        return jsonify(access_token=token), 200
    return jsonify(msg='Credenciais inválidas'), 401


@app.route('/users', methods=['POST'])
@jwt_required()
def create_user():
    """
    Cria um novo usuário.

    Requisição:
        Método: POST
        URL: /users
        Cabeçalho: Authorization: Bearer <token>
        Corpo (JSON):
        {
            "nome": "Nome do Usuário",
            "email": "email@exemplo.com"
        }

    Respostas:
        201 Created:
            - Retorna o usuário criado em formato JSON.

        400 Bad Request:
            - Caso os campos obrigatórios estejam faltando ou inválidos.

    Observações:
        - Requer autenticação JWT.
        - Os campos 'nome' e 'email' são obrigatórios.
    """
    data = request.get_json()
    user = User(nome=data['nome'], email=data['email'])
    db.session.add(user)
    db.session.commit()
    return jsonify(user.to_dict()), 201


@app.route('/users', methods=['GET'])
@jwt_required()
def list_users():
    """
    Lista todos os usuários cadastrados.

    Requisição:
        Método: GET
        URL: /users
        Cabeçalho: Authorization: Bearer <token>

    Respostas:
        200 OK:
            - Retorna uma lista de usuários em formato JSON.

    Observações:
        - Requer autenticação JWT.
        - Retorna uma lista vazia se não houver usuários.
    """
    users = User.query.all()
    return jsonify([u.to_dict() for u in users]), 200


@app.route('/users/<int:id>', methods=['GET'])
@jwt_required()
def get_user(id):
    """
    Recupera os detalhes de um usuário específico pelo ID.

    Requisição:
        Método: GET
        URL: /users/<id>
        Cabeçalho: Authorization: Bearer <token>

    Parâmetros de rota:
        id (int): ID do usuário a ser consultado.

    Respostas:
        200 OK:
            - Retorna os dados do usuário em formato JSON.

        404 Not Found:
            - Caso o usuário com o ID especificado não exista.

    Observações:
        - Requer autenticação JWT.
    """
    user = User.query.get_or_404(id)
    return jsonify(user.to_dict()), 200


@app.route('/users/<int:id>', methods=['PUT'])
@jwt_required()
def update_user(id):
    """
    Atualiza os dados de um usuário existente.

    Requisição:
        Método: PUT
        URL: /users/<id>
        Cabeçalho: Authorization: Bearer <token>
        Corpo (JSON):
        {
            "nome": "Novo Nome (opcional)",
            "email": "Novo Email (opcional)"
        }

    Parâmetros de rota:
        id (int): ID do usuário a ser atualizado.

    Respostas:
        200 OK:
            - Retorna o usuário atualizado em formato JSON.

        404 Not Found:
            - Caso o usuário com o ID especificado não exista.

    Observações:
        - Requer autenticação JWT.
        - Qualquer campo ausente permanecerá com o valor atual.
    """
    user = User.query.get_or_404(id)
    data = request.get_json()
    user.nome = data.get('nome', user.nome)
    user.email = data.get('email', user.email)
    db.session.commit()
    return jsonify(user.to_dict()), 200


@app.route('/users/<int:id>', methods=['DELETE'])
@jwt_required()
def delete_user(id):
    """
    Exclui um usuário específico.

    Requisição:
        Método: DELETE
        URL: /users/<id>
        Cabeçalho: Authorization: Bearer <token>

    Parâmetros de rota:
        id (int): ID do usuário a ser excluído.

    Respostas:
        204 No Content:
            - Indica que o usuário foi excluído com sucesso.

        404 Not Found:
            - Caso o usuário com o ID especificado não exista.

    Observações:
        - Requer autenticação JWT.
        - Após a exclusão, o recurso deixa de existir.
    """
    user = User.query.get_or_404(id)
    db.session.delete(user)
    db.session.commit()
    return '', 204


@app.route('/predict', methods=['POST'])
@jwt_required()
def predict():
    """
    Realiza uma predição numérica utilizando o modelo de IA treinado (carregado via pickle).

    Requisição:
        Método: POST
        URL: /predict
        Cabeçalho: Authorization: Bearer <token>
        Corpo (JSON):
        {
            "entrada": [valor1, valor2, ...]
        }

    Resposta:
        200 OK:
        {
            "predicao": [resultado1, resultado2, ...]
        }

        400 Bad Request:
        - Caso o JSON esteja mal formatado ou falte o campo 'entrada'.

    Observações:
        - O modelo utilizado foi previamente treinado e salvo em 'modelo_regressao.pkl'.
        - A entrada deve ser uma lista de valores numéricos.
    """
    try:
        data = request.get_json()

        # Validação da entrada
        if not data or 'entrada' not in data:
            return jsonify({"error": "Formato de entrada inválido. Exemplo: {'entrada': [5]}"}), 400

        # Prepara os dados de entrada no formato esperado pelo modelo
        entrada = np.array(data['entrada']).reshape(-1, 1)

        # Faz a predição
        resultado = modelo_regressao.predict(entrada)

        # Retorna o resultado como lista JSON serializável
        return jsonify({'predicao': resultado.tolist()}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500



In [ ]:
# Rodar o servidor Flask (roda até você parar manualmente o kernel)
app.run(port=5000)
